<a target="_blank" href="https://colab.research.google.com/github/FlyingPumba/circuits-benchmark/blob/main/DEMO_InterpBench.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Code to setup InterpBench

In [1]:
import os
import sys
!apt-get update -q && apt-get install -y --no-install-recommends libgl1-mesa-glx graphviz graphviz-dev git-lfs

zsh:1: command not found: apt-get


In [2]:
!git clone --recurse-submodules https://github.com/FlyingPumba/circuits-benchmark.git
%cd circuits-benchmark
!pip install -e .

Cloning into 'circuits-benchmark'...
remote: Enumerating objects: 4667, done.
remote: Counting objects: 100% (838/838), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 4667 (delta 579), reused 796 (delta 552), pack-reused 3829
Receiving objects: 100% (4667/4667), 19.30 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (3271/3271), done.
Submodule 'submodules/Automatic-Circuit-Discovery' (https://github.com/FlyingPumba/Automatic-Circuit-Discovery.git) registered for path 'submodules/Automatic-Circuit-Discovery'
Submodule 'submodules/iit' (https://github.com/cybershiptrooper/iit.git) registered for path 'submodules/iit'
Submodule 'submodules/tracr' (https://github.com/FlyingPumba/tracr.git) registered for path 'submodules/tracr'
Cloning into '/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/circuits-benchmark/submodules/Automatic-Circuit-Discovery'...
remote: Enumerating objects: 9802, done.        
remote: Counting objects: 100% (4036/4036), done

/Users/cybershiptrooper/src/interpretability/MATS/.iit/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Obtaining file:///Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/circuits-benchmark
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/FlyingPumba/auto-circuit.git to /private/var/folders/_k/_46xyqdj165bdcyw79k6758w0000gn/T/pip-install-rxied_6v/auto-circuit_17bad68c6f7443fdbdacefd732f72115
  Running command git clone --filter=blob:none --quiet https://github.com/FlyingPumba/auto-circuit.git /private/var/folders/_k/_46xyqdj165bdcyw79k6758w0000gn/T/pip-install-rxied_6v/auto-circuit_17bad68c6f7443fdbdacefd732f72115
  Resolved https://github.com/FlyingPumba/auto-circuit.git to commit 0df108df03533690d8d161529f823ba26b00af3a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

### Load model from InterpBench

In [30]:
%load_ext autoreload
%autoreload 2

import pickle
import torch
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

task = case_3.Case3()
task_idx = task.get_name()

In [5]:
from circuits_benchmark.utils.ll_model_loader.ll_model_loader_factory import get_ll_model_loader

ll_model_loader = get_ll_model_loader(task, interp_bench=True)
hl_ll_corr, model = ll_model_loader.load_ll_model_and_correspondence(device='cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# turn off grads
model.eval()
model.requires_grad_(False)
torch.set_grad_enabled(False)

### Run evaluations

In [31]:
# load high level model
from circuits_benchmark.utils.iit.iit_hl_model import IITHLModel
from circuits_benchmark.transformers.hooked_tracr_transformer import HookedTracrTransformer

# Prepare HL model
hl_model = task.get_hl_model()
if isinstance(hl_model, HookedTracrTransformer):
    hl_model = IITHLModel(hl_model, eval_mode=True)

# Build model pair
model_pair = task.build_model_pair(
  ll_model=model,
  hl_model=hl_model,
)

Moving model to device:  cpu
{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': HookPoint(), 'blocks.1.attn.hook_

In [8]:
from circuits_benchmark.commands.evaluation.iit.iit_eval import get_node_effects

max_len = 100 # make this smaller or larger to include more or less datapoints
node_effects, eval_metrics = get_node_effects(case=task, model_pair=model_pair, max_len=max_len, use_mean_cache=True)

100%|██████████| 1/1 [00:00<00:00, 104.11it/s]


In [9]:
node_effects

,node,status,resample_ablate_effect,mean_ablate_effect
0,"blocks.0.attn.hook_result, head 0",not_in_circuit,0.0,0.0
1,"blocks.0.attn.hook_result, head 1",not_in_circuit,0.0,0.0
2,"blocks.0.attn.hook_result, head 2",not_in_circuit,0.0,0.0
3,"blocks.0.attn.hook_result, head 3",not_in_circuit,0.0,0.0
4,"blocks.1.attn.hook_result, head 0",not_in_circuit,0.0,0.0
5,"blocks.1.attn.hook_result, head 1",not_in_circuit,0.0,0.0
6,"blocks.1.attn.hook_result, head 3",not_in_circuit,0.0,0.0
7,blocks.1.mlp.hook_post,not_in_circuit,0.0,0.0
8,blocks.0.mlp.hook_post,in_circuit,1.0,0.0
9,"blocks.1.attn.hook_result, head 2",in_circuit,1.0,0.0


In [10]:
print(eval_metrics)

val/iit_loss: 0.0001
val/IIA: 100.00%
val/accuracy: 100.00%
val/strict_accuracy: 100.00%


### Run ACDC

In [32]:
from circuits_benchmark.commands.algorithms.acdc import ACDCRunner, ACDCConfig

config = ACDCConfig(threshold=0.01)
acdc_runner = ACDCRunner(task, config=config)
acdc_runner.config

ACDCConfig(threshold=0.01, data_size=1000, include_mlp=False, next_token=False, use_pos_embed=False, indices_mode=('reverse',), names_mode=('normal',), torch_num_threads=0, max_num_epochs=100000, single_step=False, abs_value_threshold=False, online_cache_cpu=True, corrupted_cache_cpu=True, zero_ablation=False, using_wandb=False, wandb_entity_name=None, wandb_group_name=None, wandb_project_name=None, wandb_run_name=None, wandb_dir='/tmp/wandb', wandb_mode='online', seed=42, output_dir='/home/ivan/src/circuits-benchmark/results', same_size=False, device='cpu', testing=False)

In [33]:
# Prepare data
data_size = acdc_runner.config.data_size
clean_inputs = task.get_clean_data(max_samples=data_size).get_inputs()
corrupted_inputs = task.get_corrupted_data(max_samples=data_size).get_inputs()

# Prepare validation metric
metric_name = "kl" if model_pair.hl_model.is_categorical() else "l2"
validation_metric = task.get_validation_metric(model, clean_inputs, metric_name=metric_name)

output_dir = "./results/acdc_3/weight_100_100_40/threshold_0.025"
os.makedirs(output_dir, exist_ok=True)

acdc_circuit = acdc_runner.run(
    tl_model=model,
    clean_dataset=clean_inputs,
    corrupt_dataset=corrupted_inputs,
    validation_metric=validation_metric,
    output_dir=output_dir
)

dict_keys(['blocks.1.hook_resid_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_mlp_in', 'blocks.1.attn.hook_result', 'blocks.1.attn.hook_q', 'blocks.1.hook_q_input', 'blocks.1.attn.hook_k', 'blocks.1.hook_k_input', 'blocks.1.attn.hook_v', 'blocks.1.hook_v_input', 'blocks.0.hook_mlp_out', 'blocks.0.hook_mlp_in', 'blocks.0.attn.hook_result', 'blocks.0.attn.hook_q', 'blocks.0.hook_q_input', 'blocks.0.attn.hook_k', 'blocks.0.hook_k_input', 'blocks.0.attn.hook_v', 'blocks.0.hook_v_input', 'blocks.0.hook_resid_pre'])
blocks.1.hook_resid_post
blocks.1.hook_mlp_out
blocks.1.mlp.hook_post
blocks.1.mlp.hook_pre
blocks.1.hook_mlp_in
blocks.1.hook_resid_mid
blocks.1.hook_attn_out
blocks.1.attn.hook_result
blocks.1.attn.hook_z
blocks.1.attn.hook_pattern
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_v
blocks.1.attn.hook_k
blocks.1.attn.hook_q
blocks.1.hook_v_input
blocks.1.hook_k_input
blocks.1.hook_q_input
blocks.1.hook_resid_pre
blocks.0.hook_resid_post
blocks.0.hook_mlp_out
blocks.0.mlp.hook_

/home/ivan/.cache/pypoetry/virtualenvs/circuits-benchmark-EYY1cbYR-py3.11/lib/python3.11/site-packages/acdc/TLACDCExperiment.py:897: UserWarning: Finished iterating
  warnings.warn("Finished iterating")


In [34]:
print(acdc_circuit)
print(f"Nodes: {acdc_circuit.nodes}")
print(f"Edges: {acdc_circuit.edges}")

Circuit with 7 nodes and 4 edges
Nodes: [blocks.1.attn.hook_result[2], blocks.1.hook_resid_post, blocks.1.hook_v_input[2], blocks.1.attn.hook_v[2], blocks.0.hook_mlp_out, blocks.0.hook_resid_pre, blocks.0.hook_mlp_in]
Edges: [(blocks.1.attn.hook_result[2], blocks.1.hook_resid_post), (blocks.1.hook_v_input[2], blocks.1.attn.hook_v[2]), (blocks.0.hook_mlp_out, blocks.1.hook_v_input[2]), (blocks.0.hook_resid_pre, blocks.0.hook_mlp_in)]


In [35]:
# load edges from InterpBench
from circuits_benchmark.utils.circuit.circuit_eval import calculate_fpr_and_tpr, build_from_acdc_correspondence
from circuits_benchmark.utils.circuit.circuit import Circuit
from acdc.TLACDCCorrespondence import TLACDCCorrespondence
from huggingface_hub import hf_hub_download

# We artificially add the following modules to avoid import errors, since some of the files were pickled before the project was refactored 
sys.modules["circuits_benchmark.transformers.circuit"] = sys.modules["circuits_benchmark.utils.circuit.circuit"]
sys.modules["circuits_benchmark.transformers.circuit_node"] = sys.modules["circuits_benchmark.utils.circuit.circuit_node"]

def get_tpr_fpr_for_acdc_circuit(acdc_circuit: Circuit):
    edges_file = hf_hub_download("cybershiptrooper/InterpBench", subfolder=task_idx, filename="edges.pkl")
    edges = pickle.load(open(edges_file, "rb"))

    gt_circuit = Circuit()
    for edge in edges:
        gt_circuit.add_edge(edge[0], edge[1])

    full_corr = TLACDCCorrespondence.setup_from_model(model, use_pos_embed=True)
    full_circuit = build_from_acdc_correspondence(corr=full_corr)

    return calculate_fpr_and_tpr(acdc_circuit, gt_circuit, full_circuit)

get_tpr_fpr_for_acdc_circuit(acdc_circuit)


Summary:
 - Nodes TP rate: 1.0
 - Nodes FP rate: 0.0
 - Edges TP rate: 0.3333333333333333
 - Edges FP rate: 0.0


CircuitEvalResult(nodes=CircuitEvalNodesResult(true_positive={blocks.0.hook_mlp_out, blocks.1.attn.hook_result[2], blocks.1.hook_resid_post}, false_positive=set(), false_negative=set(), true_negative={blocks.1.hook_mlp_out, blocks.1.attn.hook_result[0], blocks.0.attn.hook_result[3], blocks.1.attn.hook_result[3], blocks.0.attn.hook_result[0], blocks.1.attn.hook_result[1], blocks.0.attn.hook_result[1], blocks.0.attn.hook_result[2]}, tpr=1.0, fpr=0.0), edges=CircuitEvalEdgesResult(true_positive={(blocks.0.hook_mlp_out, blocks.1.attn.hook_result[2])}, false_positive=set(), false_negative={(hook_pos_embed, blocks.1.attn.hook_result[2]), (hook_embed, blocks.0.hook_mlp_out)}, true_negative={(blocks.0.attn.hook_result[1], blocks.1.attn.hook_result[3]), (blocks.0.attn.hook_result[1], blocks.1.attn.hook_result[0]), (blocks.0.attn.hook_result[1], blocks.1.attn.hook_result[1]), (blocks.0.attn.hook_result[2], blocks.1.attn.hook_result[3]), (blocks.0.attn.hook_result[2], blocks.1.attn.hook_result[0]